In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark= SparkSession. \
builder. \
appName("week_7_assignment_two"). \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/itv009033/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

# 2. Consider a scenario where you have a large dataset (/public/trendytech/datasets/hotel_data.csv) in HDFS. Design a caching mechanism using spark external tables to improve the query performance on this dataset. The dataset contains the following columns: booking_id,guest_name, checkin_date, checkout_date, room_type, and total_price.

## A) Write a query to fetch the total count of hotel bookings in the hotel_bookings table and compare the duration it took to determine the impact of caching.

In [3]:
spark.sql("create database if not exists itv009033_hotel")

""


In [4]:
spark.sql("use itv009033_hotel")

""


In [5]:
spark.sql("create table if not exists itv009033_hotel.hotel_ext(booking_id int,guest_name string, checkin_date date, checkout_date date, room_type string, total_price double) using csv location '/public/trendytech/datasets/hotel_data.csv' ")

""


In [6]:
spark.sql("select * from itv009033_hotel.hotel_ext").show()

+----------+-----------------+------------+-------------+---------+-----------+
|booking_id|       guest_name|checkin_date|checkout_date|room_type|total_price|
+----------+-----------------+------------+-------------+---------+-----------+
|         1|         John Doe|  2023-05-01|   2023-05-05| Standard|      400.0|
|         2|       Jane Smith|  2023-05-02|   2023-05-06|   Deluxe|      600.0|
|         3|     Mark Johnson|  2023-05-03|   2023-05-08| Standard|      450.0|
|         4|     Sarah Wilson|  2023-05-04|   2023-05-07|Executive|      750.0|
|         5|      Emily Brown|  2023-05-06|   2023-05-09|   Deluxe|      550.0|
|         6|    Michael Davis|  2023-05-07|   2023-05-10| Standard|      400.0|
|         7|Samantha Thompson|  2023-05-08|   2023-05-12|   Deluxe|      600.0|
|         8|      William Lee|  2023-05-10|   2023-05-13| Standard|      450.0|
|         9|    Amanda Harris|  2023-05-11|   2023-05-16|Executive|      750.0|
|        10|  David Rodriguez|  2023-05-

In [8]:
hotels_schema = "booking_id int,guest_name string, checkin_date date, checkout_date date, room_type string, total_price double"

In [9]:
hotels = spark.read.format("csv").schema(hotels_schema).load("/public/trendytech/datasets/hotel_data.csv")

In [10]:
hotels.show()

+----------+-----------------+------------+-------------+---------+-----------+
|booking_id|       guest_name|checkin_date|checkout_date|room_type|total_price|
+----------+-----------------+------------+-------------+---------+-----------+
|         1|         John Doe|  2023-05-01|   2023-05-05| Standard|      400.0|
|         2|       Jane Smith|  2023-05-02|   2023-05-06|   Deluxe|      600.0|
|         3|     Mark Johnson|  2023-05-03|   2023-05-08| Standard|      450.0|
|         4|     Sarah Wilson|  2023-05-04|   2023-05-07|Executive|      750.0|
|         5|      Emily Brown|  2023-05-06|   2023-05-09|   Deluxe|      550.0|
|         6|    Michael Davis|  2023-05-07|   2023-05-10| Standard|      400.0|
|         7|Samantha Thompson|  2023-05-08|   2023-05-12|   Deluxe|      600.0|
|         8|      William Lee|  2023-05-10|   2023-05-13| Standard|      450.0|
|         9|    Amanda Harris|  2023-05-11|   2023-05-16|Executive|      750.0|
|        10|  David Rodriguez|  2023-05-

In [12]:
hotels.count()

107

In [13]:
spark.sql("cache lazy table itv009033_hotel.hotel_ext")

""


In [17]:
spark.sql("select count(*) from itv009033_hotel.hotel_ext").show()

+--------+
|count(1)|
+--------+
|     107|
+--------+



In [18]:
cached = hotels.cache()

In [20]:
cached.count()

107

In [21]:
from pyspark.sql.functions import year, month
from pyspark.sql.functions import countDistinct
from pyspark.storagelevel import StorageLevel

In [22]:
hotels_persist = hotels.persist(StorageLevel.MEMORY_AND_DISK_SER)

In [25]:
hotels_persist.count()

107

In [26]:
hotels_persist.show()

+----------+-----------------+------------+-------------+---------+-----------+
|booking_id|       guest_name|checkin_date|checkout_date|room_type|total_price|
+----------+-----------------+------------+-------------+---------+-----------+
|         1|         John Doe|  2023-05-01|   2023-05-05| Standard|      400.0|
|         2|       Jane Smith|  2023-05-02|   2023-05-06|   Deluxe|      600.0|
|         3|     Mark Johnson|  2023-05-03|   2023-05-08| Standard|      450.0|
|         4|     Sarah Wilson|  2023-05-04|   2023-05-07|Executive|      750.0|
|         5|      Emily Brown|  2023-05-06|   2023-05-09|   Deluxe|      550.0|
|         6|    Michael Davis|  2023-05-07|   2023-05-10| Standard|      400.0|
|         7|Samantha Thompson|  2023-05-08|   2023-05-12|   Deluxe|      600.0|
|         8|      William Lee|  2023-05-10|   2023-05-13| Standard|      450.0|
|         9|    Amanda Harris|  2023-05-11|   2023-05-16|Executive|      750.0|
|        10|  David Rodriguez|  2023-05-

## B) Calculate the average total price of bookings grouped by room_type for the first 100 records in the hotel_bookings table without caching. Execute the same query after caching the table and compare the duration

In [32]:
price = hotels_persist.groupBy("room_type").sum("total_price").withColumnRenamed("sum(total_price)", "price").cache()

In [34]:
price.show()

+---------+-------+
|room_type|  price|
+---------+-------+
|Executive|15000.0|
|   Deluxe|24750.0|
| Standard|18700.0|
+---------+-------+



In [37]:
spark.sql("select room_type, sum(total_price) as price from itv009033_hotel.hotel_ext group by 1").show()

+---------+-------+
|room_type|  price|
+---------+-------+
|Executive|15000.0|
|   Deluxe|24750.0|
| Standard|18700.0|
+---------+-------+



In [38]:
spark.sql("clear cache")

""


In [39]:
spark.stop()